In [1]:
import collections
import math
import os
import sys
import argparse
import random
from tempfile import gettempdir
import zipfile

import numpy as np
import cPickle
import random
import math
import h5py
from multiprocessing import cpu_count
import scipy.io as sio
from sklearn.preprocessing import MultiLabelBinarizer
from collections import defaultdict
from gensim.models import Word2Vec, KeyedVectors
from six import iteritems
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.utils import shuffle as skshuffle
from numpy import genfromtxt

from collections import defaultdict as dd
import tensorflow as tf
from tensorflow.python.ops.init_ops import glorot_uniform_initializer, RandomUniform, RandomNormal

from keras import layers
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten
from keras.layers.embeddings import Embedding
from keras.models import Model
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

from gensim.models import Word2Vec
from gensim.models.word2vec import Vocab

import word2vec

np.random.seed(1)
random.seed(1)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Using TensorFlow backend.


In [2]:
# Initialize parameters
DATASET = 'blogcatalog'

embedding_size = 128
learning_rate = 0.1
gl_learning_rate = 0.1
batch_size = 200
neg_samp = 0
model_file = 'trans.model'

window_size = 10
path_size = 10


use_feature = True
update_emb = True
layer_loss =  True

In [3]:
file_list = ['/hdd2/graph_embedding/customized/blogcatalog.embeddings.walks.0']
dataset = genfromtxt(file_list[0], delimiter=' ')

def get_num_vacabulary(dataset):
    word_count = 0
    for d in dataset:
        word_count = max(word_count, max(d))
    return int(word_count)

vocabulary_size = get_num_vacabulary(dataset) + 1

In [4]:
words = dataset.flatten()
words = [str(int(w)) for w in words]

In [5]:
words[:10]

['1614', '5573', '4732', '1159', '448', '5704', '5382', '1008', '5395', '6958']

In [6]:
dataset[0]

array([  1614.,   5573.,   4732.,   1159.,    448.,   5704.,   5382.,
         1008.,   5395.,   6958.,   1320.,   4139.,   6814.,   5404.,
         7988.,   8384.,  10139.,   6248.,   4226.,   2829.,   9090.,
         8456.,   6892.,    738.,   3898.,   6958.,   3048.,   6796.,
         7636.,   4104.,   7450.,   1225.,   6061.,    457.,   8968.,
           35.,   1141.,   4996.,   1453.,   2979.])

In [7]:
def build_dataset(words):
    count = []
#     count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = int(word) #len(dictionary)
    data = list()
    for word in words:
        if word in dictionary:
            index = dictionary[word]
#         else:
#             index = 0  # dictionary['UNK']
        data.append(index)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
    return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])

('Most common words (+UNK)', [('4838', 196735), ('175', 195187), ('4373', 169490), ('8156', 147968), ('1225', 137221)])
('Sample data', [1614, 5573, 4732, 1159, 448, 5704, 5382, 1008, 5395, 6958])


In [8]:
print(data[:21])
print(dataset[0][:21])

[1614, 5573, 4732, 1159, 448, 5704, 5382, 1008, 5395, 6958, 1320, 4139, 6814, 5404, 7988, 8384, 10139, 6248, 4226, 2829, 9090]
[  1614.   5573.   4732.   1159.    448.   5704.   5382.   1008.   5395.
   6958.   1320.   4139.   6814.   5404.   7988.   8384.  10139.   6248.
   4226.   2829.   9090.]


In [9]:
dictionary['1008']

1008

In [10]:
data_index = 0

def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1 # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
#     print(buffer)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [ skip_window ]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels

# print('data:', [reverse_dictionary[di] for di in data[:8]])

for num_skips, skip_window in [(20, 10)]:
    data_index = 0
    batch, labels = generate_batch(batch_size=20, num_skips=num_skips, skip_window=skip_window)
    print('\nwith num_skips = %d and skip_window = %d:' % (num_skips, skip_window))
    print('    batch node id:', [reverse_dictionary[bi] for bi in batch])
    print('    node position:')
    print(batch)
    print(labels)
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(20)])


with num_skips = 20 and skip_window = 10:
('    batch node id:', ['1320', '1320', '1320', '1320', '1320', '1320', '1320', '1320', '1320', '1320', '1320', '1320', '1320', '1320', '1320', '1320', '1320', '1320', '1320', '1320'])
    node position:
[1320 1320 1320 1320 1320 1320 1320 1320 1320 1320 1320 1320 1320 1320 1320
 1320 1320 1320 1320 1320]
[[ 4732]
 [ 6248]
 [10139]
 [ 5704]
 [ 6958]
 [ 5404]
 [ 5573]
 [ 1614]
 [ 8384]
 [  448]
 [ 2829]
 [ 4226]
 [ 4139]
 [ 5395]
 [ 5382]
 [ 1159]
 [ 9090]
 [ 7988]
 [ 6814]
 [ 1008]]
('    labels:', ['4732', '6248', '10139', '5704', '6958', '5404', '5573', '1614', '8384', '448', '2829', '4226', '4139', '5395', '5382', '1159', '9090', '7988', '6814', '1008'])


In [11]:
reverse_dictionary[0]

'0'

In [12]:
# inject trained embedding
model = word2vec.load('/hdd2/graph_embedding/tmp/blogcatalog.embeddings.walks.0.bin')
features_matrix = np.asarray([model[str(node)] for node in range(len(model.vocab)-1)])
# make the order consistant to the current one
reordered_list = np.asarray([int(reverse_dictionary[idx]) for idx in range(len(model.vocab)-1)])
reordered_features_matrix = features_matrix[reordered_list]

In [41]:
# Reproduce Gensim weights initialization
def seeded_vector(seed_string, vector_size):
    """Create one 'random' vector (but deterministic by seed_string)"""
    # Note: built-in hash() may vary by Python version or even (in Py3.x) per launch
    once = np.random.RandomState(hash(seed_string) & 0xffffffff)
    return (once.rand(vector_size) - 0.5) / vector_size

features_list = []
for idx in range(vocabulary_size):
    str_node = reverse_dictionary[idx]
    features_list.append(seeded_vector(str_node + str(1), 128))
features_matrix = np.asarray(features_list)

In [45]:
batch_size = 2000
embedding_size = 128 # Dimension of the embedding vector.
skip_window = 10 # How many words to consider left and right.
num_skips = 20 # How many times to reuse an input to generate a label.
# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. 
valid_size = 16 # Random set of words to evaluate similarity on.
valid_window = 100 # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size))
num_sampled = 64 # Number of negative examples to sample.

config = tf.ConfigProto(allow_soft_placement=True)
graph = tf.Graph()

with graph.as_default(), tf.device('/gpu:0'):

    # Input data.
    train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Variables.
#     embeddings = tf.Variable(
#         tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    with tf.device('/cpu:0'):
        embeddings = tf.Variable(features_matrix, dtype=tf.float32)
    softmax_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                         stddev=1.0 / math.sqrt(embedding_size)))
    softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # Model.
    # Look up embeddings for inputs.
    embed = tf.nn.embedding_lookup(embeddings, train_dataset)
    # Compute the softmax loss, using a sample of the negative labels each time.
    loss = tf.reduce_mean(
        tf.nn.sampled_softmax_loss(weights=softmax_weights, biases=softmax_biases, inputs=embed,
                                   labels=train_labels, num_sampled=num_sampled, num_classes=vocabulary_size))

    # Optimizer.
    # Note: The optimizer will optimize the softmax_weights AND the embeddings.
    # This is because the embeddings are defined as a variable quantity and the
    # optimizer's `minimize` method will by default modify all variable quantities 
    # that contribute to the tensor it is passed.
    # See docs on `tf.train.Optimizer.minimize()` for more details.
    optimizer = tf.train.AdagradOptimizer(learning_rate=0.25).minimize(loss)

    # Compute the similarity between minibatch examples and all embeddings.
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(
        normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))

In [46]:
with tf.Session(graph=graph, config=config) as session:
    tf.global_variables_initializer().run()
    batch_data, batch_labels = generate_batch(
        batch_size, num_skips, skip_window)
    feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
    _, l, res_embed = session.run([optimizer, loss, embed], feed_dict=feed_dict)
    print('loss: %.4f' %(l))

loss: 3.4940


In [ ]:
num_steps = 50000001

with tf.Session(graph=graph, config=config) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    average_loss = 0
    for step in range(num_steps):
        batch_data, batch_labels = generate_batch(
            batch_size, num_skips, skip_window)
        feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
#         print('---------%d-----------' %step)
#         print(batch_data)
#         print(batch_labels)
        _, l = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += l
#         print(step)
        if step % 2000 == 0:
            if step > 0:
                average_loss = average_loss / 2000.0
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step %d: %f' % (step, average_loss))
            average_loss = 0
        # note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            embedding_filename = '/hdd2/graph_embedding/customized/results/deepwalk_unsupervised/blog_embeddings_iter%d.txt' %step
            not_normal_embeddings = embeddings.eval()
#             ordered_embeddings = [not_normal_embeddings[dictionary[str(node)]] for node in range(len(dictionary))]
            np.savetxt(embedding_filename, not_normal_embeddings)
#             sim = similarity.eval()
#             for i in range(valid_size):
#                 valid_word = reverse_dictionary[valid_examples[i]]
#                 top_k = 8 # number of nearest neighbors
#                 nearest = (-sim[i, :]).argsort()[1:top_k+1]
#                 log = 'Nearest to %s:' % valid_word
#                 for k in range(top_k):
#                     close_word = reverse_dictionary[nearest[k]]
#                     log = '%s %s,' % (log, close_word)
#                 print(log)
    final_embeddings = normalized_embeddings.eval()
    not_normal_embeddings = embeddings.eval()

Initialized
Average loss at step 0: 3.566697
Average loss at step 2000: 2.948955
Average loss at step 4000: 2.668442
Average loss at step 6000: 2.551956
Average loss at step 8000: 2.509967
Average loss at step 10000: 2.460560
Average loss at step 12000: 2.430992
Average loss at step 14000: 2.445639
Average loss at step 16000: 2.425408
Average loss at step 18000: 2.397098
Average loss at step 20000: 2.387706
Average loss at step 22000: 2.384186
Average loss at step 24000: 2.367495
Average loss at step 26000: 2.378719
Average loss at step 28000: 2.358239
Average loss at step 30000: 2.363578
Average loss at step 32000: 2.341658
Average loss at step 34000: 2.360979
Average loss at step 36000: 2.344888
Average loss at step 38000: 2.351796
Average loss at step 40000: 2.334078
Average loss at step 42000: 2.351195
Average loss at step 44000: 2.359530
Average loss at step 46000: 2.353113
Average loss at step 48000: 2.329995
Average loss at step 50000: 2.335786
Average loss at step 52000: 2.3392

Average loss at step 434000: 2.312875
Average loss at step 436000: 2.299097
Average loss at step 438000: 2.322914
Average loss at step 440000: 2.298892
Average loss at step 442000: 2.307166
Average loss at step 444000: 2.313777
Average loss at step 446000: 2.304706
Average loss at step 448000: 2.315521
Average loss at step 450000: 2.307829
Average loss at step 452000: 2.326917
Average loss at step 454000: 2.318991
Average loss at step 456000: 2.318965
Average loss at step 458000: 2.333807
Average loss at step 460000: 2.304068
Average loss at step 462000: 2.310594
Average loss at step 464000: 2.306950
Average loss at step 466000: 2.296040
Average loss at step 468000: 2.303904
Average loss at step 470000: 2.298918
Average loss at step 472000: 2.314283
Average loss at step 474000: 2.303488
Average loss at step 476000: 2.316712
Average loss at step 478000: 2.319943
Average loss at step 480000: 2.325993
Average loss at step 482000: 2.296346
Average loss at step 484000: 2.302853
Average loss

In [35]:
np.unique(batch_data)

array([   40,    81,   175,   222,   394,   666,   738,   749,   766,
         813,  1008,  1019,  1290,  1507,  1555,  1643,  1713,  1810,
        1839,  2002,  2520,  2636,  2927,  2998,  3091,  3140,  3146,
        3197,  3319,  3333,  3406,  3560,  3690,  3753,  3783,  3796,
        3907,  3909,  3923,  4310,  4373,  4490,  4531,  4612,  4614,
        4651,  4719,  4723,  4732,  5067,  5090,  5150,  5240,  5264,
        5285,  5378,  5449,  5624,  5657,  5767,  5783,  5792,  5859,
        6073,  6250,  6251,  6372,  6434,  6495,  6546,  6550,  6598,
        6872,  6935,  6958,  6985,  7132,  7392,  7563,  7636,  7708,
        8002,  8054,  8378,  8580,  8614,  8755,  8858,  8910,  9339,
        9419, 10172, 10215, 10311], dtype=int32)

In [36]:
np.unique(batch_labels)

array([   40,    81,   175,   222,   393,   394,   596,   666,   738,
         749,   766,   813,  1008,  1019,  1290,  1346,  1507,  1555,
        1643,  1713,  1810,  1839,  2002,  2520,  2636,  2927,  2998,
        3091,  3140,  3146,  3197,  3319,  3333,  3406,  3560,  3690,
        3753,  3783,  3796,  3907,  3909,  3923,  4310,  4373,  4490,
        4531,  4612,  4614,  4638,  4651,  4662,  4719,  4723,  4732,
        4804,  5067,  5090,  5150,  5240,  5264,  5285,  5378,  5449,
        5624,  5657,  5767,  5783,  5792,  5859,  6073,  6250,  6251,
        6372,  6434,  6495,  6546,  6550,  6598,  6707,  6872,  6935,
        6958,  6985,  7132,  7329,  7356,  7392,  7563,  7636,  7708,
        8002,  8054,  8199,  8378,  8580,  8614,  8755,  8774,  8858,
        8910,  9187,  9339,  9419,  9635,  9859,  9889, 10172, 10215, 10311], dtype=int32)

In [325]:
embedding_filename = '/hdd2/graph_embedding/customized/blog_embeddings_1.txt'
ordered_embeddings = [final_embeddings[dictionary[str(node)]] for node in range(len(dictionary))]
np.savetxt(embedding_filename, ordered_embeddings)

In [262]:
ordered_embeddings[1209]

array([ 58.47768402, -14.06970406, -11.84331131,   3.77727461,
        27.18597221, -25.0004673 ,  65.72541809, -65.32319641,
        46.84730148, -17.94046402,  -9.88634205,   4.96879911,
       -39.52791977, -41.69575119,  74.56922913,   7.61631155,
        23.03376389,  50.97651291, -33.59208679,  31.2266407 ,
        81.93914032,  15.72993279,  -5.28370953,  17.18107605,
       -75.29068756,  14.2514925 ,   7.94441748,  -4.89663982,
       -51.30142212, -33.08482361, -32.42542267,   1.93974102,
        46.80765533,   8.50759125,  10.44501019, -40.15472794,
        -5.02595329, -36.50287247, -45.88523102, -79.72190094,
       -17.26217461,   8.80545712, -52.35463333, -16.01053047,
       -43.30438232, -39.79665756, -16.72794151,  34.34871674,
        57.16449738,  -4.06061411, -72.57341003,  -7.9510541 ,
        63.76834869,  43.05852509,   2.076653  , -98.20155334,
        16.07836723,  46.57059479,  68.2883606 ,  -1.08950758,
         5.00839376,   9.26826859,   8.00768375,  18.40

In [263]:
reverse_dictionary[1209]

'0'

In [259]:
embeddings = np.loadtxt(embedding_filename)

In [260]:
embeddings[1209]

array([ 58.47768402, -14.06970406, -11.84331131,   3.77727461,
        27.18597221, -25.0004673 ,  65.72541809, -65.32319641,
        46.84730148, -17.94046402,  -9.88634205,   4.96879911,
       -39.52791977, -41.69575119,  74.56922913,   7.61631155,
        23.03376389,  50.97651291, -33.59208679,  31.2266407 ,
        81.93914032,  15.72993279,  -5.28370953,  17.18107605,
       -75.29068756,  14.2514925 ,   7.94441748,  -4.89663982,
       -51.30142212, -33.08482361, -32.42542267,   1.93974102,
        46.80765533,   8.50759125,  10.44501019, -40.15472794,
        -5.02595329, -36.50287247, -45.88523102, -79.72190094,
       -17.26217461,   8.80545712, -52.35463333, -16.01053047,
       -43.30438232, -39.79665756, -16.72794151,  34.34871674,
        57.16449738,  -4.06061411, -72.57341003,  -7.9510541 ,
        63.76834869,  43.05852509,   2.076653  , -98.20155334,
        16.07836723,  46.57059479,  68.2883606 ,  -1.08950758,
         5.00839376,   9.26826859,   8.00768375,  18.40

In [150]:
np.unique(dictionary.values())

array([    0,     1,     2, ..., 10308, 10309, 10310])

In [152]:
np.unique(dataset.flatten())

array([  0.00000000e+00,   1.00000000e+00,   2.00000000e+00, ...,
         1.03090000e+04,   1.03100000e+04,   1.03110000e+04])

In [153]:
vocabulary_size

10312

In [155]:
len(reverse_dictionary)

10311

In [270]:
from keras.preprocessing import sequence
from keras.preprocessing.sequence import skipgrams

In [273]:
sampling_table = sequence.make_sampling_table(vocabulary_size)
couples, labels = skipgrams(data, vocabulary_size, window_size=10, sampling_table=sampling_table, negative_samples=5)
# word_target, word_context = zip(*couples)
# word_target = np.array(word_target, dtype="int32")
# word_context = np.array(word_context, dtype="int32")

# print(couples[:10], labels[:10])

KeyboardInterrupt: 

In [26]:
# create some input variables
input_target = Input((1,))
input_context = Input((1,))

embedding = Embedding(vocab_size, vector_dim, input_length=1, name='embedding')

target = embedding(input_target)
target = Reshape((vector_dim, 1))(target)
context = embedding(input_context)
context = Reshape((vector_dim, 1))(context)

In [36]:
i = 0
for line in walks_corpus:
    i += 1
print(i)

824960


In [21]:
# model = Skipgram(sentences=walks_corpus, vocabulary_counts=vocabulary_size, size=128,
#                  window=10, min_count=0, trim_rule=None, workers=1)

In [31]:
1

1

In [89]:
def sparse2graph(x):
    G = defaultdict(lambda: set())
    cx = x.tocoo()
    for i,j,v in zip(cx.row, cx.col, cx.data):
        G[i].add(j)
    return {str(k): [str(x) for x in v] for k,v in iteritems(G)}

In [90]:
class TopKRanker(OneVsRestClassifier):
    def predict(self, X, top_k_list):
        assert X.shape[0] == len(top_k_list)
        probs = np.asarray(super(TopKRanker, self).predict_proba(X))
        all_labels = []
        for i, k in enumerate(top_k_list):
            probs_ = probs[i, :]
            labels = self.classes_[probs_.argsort()[-k:]].tolist()
            all_labels.append(labels)
        return all_labels

In [91]:
def scoring(emb_filename, matfile):
    # 0. Files
    embeddings_file = emb_filename

    # 1. Load Embeddings
#     embeddings = np.loadtxt(embeddings_file)
    
    ## for original deepwalk
    #model = KeyedVectors.load_word2vec_format(embeddings_file, binary=False)
    
    ## for external word2vec lib
    model = word2vec.load('/hdd2/graph_embedding/tmp/blogcatalog.embeddings.walks.0.bin')

    # 2. Load labels
    mat = sio.loadmat(matfile)
    A = mat['network']
    graph = sparse2graph(A)
    labels_matrix = mat['group']
    labels_count = labels_matrix.shape[1]
    mlb = MultiLabelBinarizer(range(labels_count))

    # Map nodes to their features (note:  assumes nodes are labeled as integers 1:N)
#     features_matrix = embeddings

#     features_matrix = np.asarray([model[str(node)] for node in range(len(graph))])

    # use other word2vec lib
    features_matrix = np.asarray([model[str(node)] for node in xrange(len(model.vocab)-1)])
    
    # 2. Shuffle, to create train/test groups
    shuffles = []
    for x in range(1):
        shuffles.append(skshuffle(features_matrix, labels_matrix))

    # 3. to score each train/test group
    all_results = defaultdict(list)

#     if args.all:
#         training_percents = numpy.asarray(range(1, 10)) * .1
#     else:
#         training_percents = [0.1, 0.5, 0.9]
    training_percents = [0.1]
    for train_percent in training_percents:
        for shuf in shuffles:
            
            X, y = shuf

            training_size = int(train_percent * X.shape[0])

            X_train = X[:training_size, :]
            y_train_ = y[:training_size]

            y_train = [[] for x in range(y_train_.shape[0])]


            cy =  y_train_.tocoo()
            for i, j in zip(cy.row, cy.col):
                y_train[i].append(j)

            assert sum(len(l) for l in y_train) == y_train_.nnz

            X_test = X[training_size:, :]
            y_test_ = y[training_size:]

            y_test = [[] for _ in range(y_test_.shape[0])]

            cy =  y_test_.tocoo()
            for i, j in zip(cy.row, cy.col):
                y_test[i].append(j)

            clf = TopKRanker(LogisticRegression())
            clf.fit(X_train, y_train_)

            # find out how many labels should be predicted
            top_k_list = [len(l) for l in y_test]
            preds = clf.predict(X_test, top_k_list)

            results = {}
            averages = ["micro", "macro"]
            for average in averages:
                results[average] = f1_score(mlb.fit_transform(y_test), mlb.fit_transform(preds), average=average)

            all_results[train_percent].append(results)

    print ('Results, using embeddings of dimensionality', X.shape[1])
    print ('-------------------')
    for train_percent in sorted(all_results.keys()):
        print ('Train percent:', train_percent)
    for index, result in enumerate(all_results[train_percent]):
        print ('Shuffle #%d:   ' % (index + 1), result)
    avg_score = defaultdict(float)
    for score_dict in all_results[train_percent]:
        for metric, score in iteritems(score_dict):
            avg_score[metric] += score
    for metric in avg_score:
        avg_score[metric] /= len(all_results[train_percent])
    print ('Average score:', dict(avg_score))
    print ('-------------------')

In [92]:
matfile = '/hdd2/graph_embedding/deepwalk/example_graphs/blogcatalog.mat'
embedding_filename = '/hdd2/graph_embedding/customized/blog_embeddings.iter2.txt'
embedding_filename_original = '/hdd2/graph_embedding/customized/model_ns5_iter1.output'
embedding_filename_other_lib = '/hdd2/graph_embedding/tmp/blogcatalog.embeddings.walks.0.bin'

scoring('', matfile)

NameError: global name 'word2vec' is not defined

In [23]:
embedding_filename_original = '/hdd2/graph_embedding/customized/model_ns5_iter1.output'
model = KeyedVectors.load_word2vec_format(embedding_filename_original, binary=False)
                                          
embedding_filename = '/hdd2/graph_embedding/customized/blog_embeddings.iter2.txt'
embeddings = np.loadtxt(embedding_filename)


In [22]:
model['0']

array([ 0.110529  ,  0.19354101,  0.3969    ,  0.16305301, -0.198276  ,
       -0.219273  ,  0.045498  ,  0.138824  , -0.174495  ,  0.20618799,
        0.48984301,  0.302856  , -0.46013999,  0.227097  ,  0.071323  ,
       -0.43010399,  0.015921  , -0.021136  ,  0.28175899, -0.26225701,
        0.102745  , -0.31963301,  0.058964  ,  0.132149  ,  0.176946  ,
       -0.155433  ,  0.160973  , -0.282166  ,  0.030017  , -0.165079  ,
        0.25168899, -0.52189702,  0.005448  ,  0.18592501, -0.013992  ,
       -0.070675  ,  0.033961  ,  0.117675  , -0.073083  ,  0.068748  ,
       -0.10755   ,  0.068676  ,  0.17162   , -0.136898  ,  0.17979699,
       -0.106551  , -0.212037  ,  0.103523  , -0.242975  , -0.46731299,
        0.182107  ,  0.092075  , -0.141946  ,  0.051342  ,  0.31653801,
        0.28085399, -0.029812  ,  0.19881   ,  0.31846499, -0.12293   ,
        0.22415499, -0.21315201, -0.220193  , -0.15813901,  0.104372  ,
       -0.038969  ,  0.020534  , -0.18385699, -0.049377  ,  0.24

In [26]:
embeddings[0]

array([ -1.47322476e-01,   2.67052501e-02,   1.52672976e-02,
        -1.37635857e-01,  -6.47586957e-02,  -2.46349778e-02,
        -8.18009302e-02,   6.91710338e-02,  -9.66271833e-02,
        -9.89247933e-02,   5.70955575e-02,   1.53366596e-01,
         8.30798447e-02,  -1.44452751e-01,   9.11711424e-04,
         1.01996966e-01,   4.97803837e-02,  -4.91250493e-02,
         1.03872299e-01,  -1.11238003e-01,   3.35200056e-02,
         6.37169033e-02,  -3.92291024e-02,  -2.92820130e-02,
        -2.85616820e-03,  -3.53677608e-02,  -5.77375665e-02,
        -3.55093554e-02,   4.43368331e-02,  -1.63600937e-01,
        -5.06901741e-02,   1.09737791e-01,  -6.03695633e-03,
        -9.09792027e-04,  -9.44678709e-02,  -5.35581484e-02,
         5.67527264e-02,  -2.60556079e-02,   1.89456977e-02,
         2.38609854e-02,   4.84210551e-02,   1.34210438e-01,
        -1.30895942e-01,   6.51589632e-02,   6.92558140e-02,
        -5.88307390e-03,  -1.87402926e-02,   1.52439341e-01,
        -7.65966475e-02,

In [7]:
mat = sio.loadmat(matfile)
A = mat['network']
labels_matrix = mat['group']
labels_count = labels_matrix.shape[1]
mlb = MultiLabelBinarizer(range(labels_count))

TypeError: 'MultiLabelBinarizer' object is not iterable

In [27]:
# import modules & set up logging
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [28]:
sentences = [['first', 'sentence'], ['second', 'sentence']]
# train word2vec on the two sentences
model = gensim.models.Word2Vec(sentences, min_count=1)

2018-03-13 21:07:11,853 : INFO : collecting all words and their counts
2018-03-13 21:07:11,855 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-03-13 21:07:11,857 : INFO : collected 3 word types from a corpus of 4 raw words and 2 sentences
2018-03-13 21:07:11,858 : INFO : Loading a fresh vocabulary
2018-03-13 21:07:11,860 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2018-03-13 21:07:11,863 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2018-03-13 21:07:11,865 : INFO : deleting the raw counts dictionary of 3 items
2018-03-13 21:07:11,868 : INFO : sample=0.001 downsamples 3 most-common words
2018-03-13 21:07:11,870 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2018-03-13 21:07:11,872 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2018-03-13 21:07:11,874 : INFO : resetting layer weights
2018-03-13 21:07:11,876 : INFO : training model with 3 workers o

In [87]:
init_iter_label = 10
init_iter_graph = 7
inst_generator = gen_train_inst()
graph_generator = gen_graph()
# Generates pairs with the same label (1) or different labels (-1)
label_generator = gen_label_graph()

In [17]:
max_iter = 1
iter_graph = 0
iter_inst = 5
iter_label = 0
use_reweight = False

In [92]:
# with tf.Session() as session:
#     gx, gy = next(label_generator)
#     tf.global_variables_initializer().run()
#     feed_dict={g_sym: gx[:, 0], gy_sym: gx[:, 1].reshape(gx.shape[0],1)}
#     _, loss = session.run([g_optimizer, g_loss], feed_dict=feed_dict)
#     print 'iter label', i, loss

In [93]:
res_reweight[0]

array([ 0.00497301], dtype=float32)

In [353]:
# with tf.Session() as session:
# #     tf.global_variables_initializer().run()
#     feed_dict={x_sym: xs, y_sym: ys}
#     ll, l = session.run([optimizer, loss], feed_dict=feed_dict)

In [250]:
import word2vec

In [15]:
word2vec.word2phrase('/hdd2/graph_embedding/tmp/text8', '/hdd2/graph_embedding/tmp/text8-phrases', verbose=True)

Starting training using file /hdd2/graph_embedding/tmp/text8
Words processed: 17000K     Vocab size: 4399K  
Vocab size (unigrams + bigrams): 2419827
Words in train file: 17005206


In [16]:
word2vec.word2vec('/hdd2/graph_embedding/tmp/text8-phrases', '/hdd2/graph_embedding/tmp/text8.bin', size=100, verbose=True)

Starting training using file /hdd2/graph_embedding/tmp/text8-phrases
Vocab size: 98331
Words in train file: 15857306
Alpha: 0.000002  Progress: 100.04%  Words/thread/sec: 467.25k  

In [251]:
word2vec.word2vec(data,
                  '/hdd2/graph_embedding/tmp/blogcatalog.embeddings.walks.1.bin', 
                  size=128, verbose=True)

OSError: [Errno 7] Argument list too long

In [22]:
model = word2vec.load('/hdd2/graph_embedding/tmp/blogcatalog.embeddings.walks.0.bin')

In [40]:
model.vocab

array([u'</s>', u'4838', u'175', ..., u'9756', u'1678', u'2931'], 
      dtype='<U78')

In [52]:
features_matrix = np.asarray([model[str(node)] for node in xrange(len(model.vocab)-1)])

In [48]:
model.vocab

array([u'</s>', u'4838', u'175', ..., u'9756', u'1678', u'2931'], 
      dtype='<U78')

In [38]:
model.vectors[1]

array([ 0.03300432, -0.07565996, -0.16463131, -0.10145742,  0.00168844,
       -0.09007662, -0.03259023, -0.22714335, -0.00282227, -0.05083675,
       -0.07032426, -0.15832779, -0.13998848,  0.07574239,  0.01739285,
        0.05170207, -0.01704509, -0.04309488,  0.13984987, -0.07098495,
       -0.08131668, -0.06108802,  0.03238546,  0.03199002, -0.14047897,
        0.1192169 , -0.11983237,  0.17141342, -0.04238614, -0.07682881,
       -0.09226497, -0.08641661, -0.05119943,  0.04880232,  0.05202007,
        0.11911941,  0.02494194,  0.0460702 , -0.02544389,  0.02227429,
       -0.02918422,  0.08963519, -0.01449389,  0.13147458,  0.04316   ,
        0.05062373,  0.01030556,  0.14064927,  0.05300813,  0.09054667,
        0.16458763,  0.00496632,  0.09853401, -0.09157445,  0.03466847,
       -0.192109  , -0.04286476,  0.0337339 ,  0.09414463,  0.02883872,
       -0.12935385,  0.05580147,  0.04444493,  0.12305645, -0.06843153,
       -0.05650309, -0.06930542, -0.0933516 , -0.08945961,  0.03

In [50]:
model['0']

array([-0.03359349, -0.01812873, -0.00173358,  0.01964512,  0.10055595,
        0.02292307,  0.0607472 ,  0.04138128, -0.12003173, -0.09744611,
        0.07093195, -0.01193107,  0.07130294,  0.08676811,  0.11344255,
        0.05344105, -0.02284033,  0.05260049, -0.01932774, -0.09711266,
       -0.19951601,  0.00738851, -0.07712144, -0.07025649, -0.03221126,
       -0.13667656,  0.08364721, -0.04125704, -0.03211425,  0.06983157,
       -0.00799658, -0.01177289, -0.16706869,  0.07463736, -0.09091619,
       -0.11201481,  0.08337475,  0.05930549, -0.07940312, -0.05631904,
       -0.01991395, -0.02349721,  0.1824768 ,  0.01132163, -0.04610476,
        0.01006832,  0.04507252, -0.08544502,  0.02825145,  0.02216966,
        0.03565323, -0.06984831,  0.03623525,  0.1129318 , -0.09319748,
       -0.04915803,  0.11094059,  0.0029751 , -0.08399934,  0.21738739,
        0.06743176, -0.09108514,  0.01798326, -0.08503798,  0.0290214 ,
        0.07596923, -0.02369601,  0.11986063, -0.04430954, -0.07

In [53]:
features_matrix[0]

array([-0.03359349, -0.01812873, -0.00173358,  0.01964512,  0.10055595,
        0.02292307,  0.0607472 ,  0.04138128, -0.12003173, -0.09744611,
        0.07093195, -0.01193107,  0.07130294,  0.08676811,  0.11344255,
        0.05344105, -0.02284033,  0.05260049, -0.01932774, -0.09711266,
       -0.19951601,  0.00738851, -0.07712144, -0.07025649, -0.03221126,
       -0.13667656,  0.08364721, -0.04125704, -0.03211425,  0.06983157,
       -0.00799658, -0.01177289, -0.16706869,  0.07463736, -0.09091619,
       -0.11201481,  0.08337475,  0.05930549, -0.07940312, -0.05631904,
       -0.01991395, -0.02349721,  0.1824768 ,  0.01132163, -0.04610476,
        0.01006832,  0.04507252, -0.08544502,  0.02825145,  0.02216966,
        0.03565323, -0.06984831,  0.03623525,  0.1129318 , -0.09319748,
       -0.04915803,  0.11094059,  0.0029751 , -0.08399934,  0.21738739,
        0.06743176, -0.09108514,  0.01798326, -0.08503798,  0.0290214 ,
        0.07596923, -0.02369601,  0.11986063, -0.04430954, -0.07